In [1]:
import numpy as np
from scipy.stats import geom, multivariate_normal

In [2]:
from scipy.optimize import minimize

In [3]:
N_sample = 1024
dim_x = 20
dim_z = 20
r = 0.6


In [4]:
def p_theta_x(theta):
    mu = theta
    covariance = 2 * np.eye(dim_x)
    return multivariate_normal(mean=mu, cov=covariance)

def p_theta_x_given_z(z):
    mu = z
    covariance = np.eye(dim_x)
    return multivariate_normal(mean=mu, cov=covariance)

def p_theta_z(theta):
    mu = theta
    covariance = np.eye(dim_z)
    return multivariate_normal(mean=mu, cov=covariance)

p = geom

def logmeanexp(data, axis=None):
    max_val = np.max(data, axis=axis)
    return max_val + np.log(np.mean(np.exp(data - max_val), axis=axis))

Simulation des observations x : on fixe theta_0 qui sera estimé par la suite. 

In [5]:
theta_0 = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20)
X = p_theta_x(theta_0).rvs(size=N_sample)
print(X[:2])

l_true = np.mean(np.log(p_theta_x(theta_0).pdf(X)))
print(l_true)

[[ 0.9273549   0.33482018  3.57706978  4.13042656  6.18264001  5.53791592
   4.77281956  8.03746989  8.87192774 10.73869364  7.80488678 13.60427996
  15.35014703 13.93989203 13.97866803 17.27583844 17.36332741 17.21515519
  20.94081452 19.30262564]
 [-0.49211995 -0.28160759  2.33667874  3.38763304  3.73595128  6.00995612
   5.19611569  9.70996431  8.91294655 10.52639965 11.88312733 13.9241664
  10.90936035 14.04677807 14.04933701 16.65732746 17.98330548 16.77511653
  19.64482665 20.53698236]]
-35.169274755844725


In [6]:
A = 1 / 2 * np.eye(dim_z, dim_x) #size = dim_z, dim_x
b = np.mean(X, axis=0) #size = dim_z, 1

In [7]:
def q_phi_z_given_x(x):
    mu = A @ x + b
    covariance = 2 / 3 * np.eye(dim_z)
    return multivariate_normal(mean=mu, cov=covariance)

In [8]:


empirical_bias_squared_ss = (l_hat_ml_ss - l_true) ** 2
print("carré du biais empirique de l'estimateur ss :", empirical_bias_squared_ss)

empirical_bias_squared_rr = (l_hat_ml_rr - l_true) ** 2
print("carré du biais empirique de l'estimateur rr :", empirical_bias_squared_rr)

empirical_bias_squared_iwae = (l_hat_iwae - l_true) ** 2
print("carré du biais empirique de l'estimateur iwae :", empirical_bias_squared_iwae)

empirical_bias_squared_sumo = (l_hat_sumo - l_true) ** 2
print("carré du biais empirique de l'estimateur sumo :", empirical_bias_squared_sumo)

NameError: name 'l_hat_ml_ss' is not defined

In [9]:
# Définition de l'estimateur 
def estimateur_ml_ss(theta, x):
    K = p.rvs(r)
    size = 2 ** (K + 1)
    Z = q_phi_z_given_x(x).rvs(size=size)
    Z_E, Z_O = Z[::2], Z[1::2]
    log_weights = [np.log(p_theta_z(theta).pdf(z) * p_theta_x_given_z(z).pdf(x) / q_phi_z_given_x(x).pdf(z)) for z in Z]
    log_weights_E, log_weights_O = log_weights[::2], log_weights[1::2]
    I_0 = np.mean(log_weights)
    l_hat_E, l_hat_O = logmeanexp(log_weights_E), logmeanexp(log_weights_O)
    l_hat_O_E = logmeanexp(log_weights)
    delta_K = l_hat_O_E - 0.5 * (l_hat_O + l_hat_E)
    l_hat_ml_ss_x = I_0 + delta_K / p(r).pmf(K)
    
    
    
    
    # Retourner l'estimation
    return l_hat_ml_ss_x

def aggregate_estimator_ml_ss(theta):
    l_hat_ml_ss_list = []

    for x in X:
        l_hat_ml_ss_list.append(estimateur_ml_ss(theta, x))
    return np.mean(l_hat_ml_ss_list)
    


In [28]:
print(aggregate_estimator_ml_ss((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20)))

/tmp/ipykernel_95902/1571981411.py:7: RuntimeWarning: divide by zero encountered in log
  log_weights = [np.log(p_theta_z(theta).pdf(z) * p_theta_x_given_z(z).pdf(x) / q_phi_z_given_x(x).pdf(z)) for z in Z]
/tmp/ipykernel_95902/217849312.py:20: RuntimeWarning: invalid value encountered in subtract
  return max_val + np.log(np.mean(np.exp(data - max_val), axis=axis))


nan


In [26]:

# Définir une fonction objectif pour l'optimisation
def objectif_ml_ss(theta):
    return -aggregate_estimator_ml_ss(theta)  # On minimise l'opposé de l'estimateur

# Supposer une valeur initiale pour theta
theta_init_ml_ss = [2, 3, 2, 5, 5, 7, 6, 8, 9, 10, 9, 13, 13, 13, 15, 16, 16, 18, 17, 20]

# Minimiser la fonction objectif pour trouver theta optimal
resultat_optimisation_ml_ss = minimize(objectif_ml_ss, theta_init_ml_ss)

# Récupérer le theta optimal trouvé par l'optimisation
theta_optimal_ml_ss = resultat_optimisation_ml_ss.x

# Imprimer le résultat
print("Theta optimal trouvé par l'optimisation:", theta_optimal_ml_ss)

/tmp/ipykernel_95902/1571981411.py:7: RuntimeWarning: divide by zero encountered in log
  log_weights = [np.log(p_theta_z(theta).pdf(z) * p_theta_x_given_z(z).pdf(x) / q_phi_z_given_x(x).pdf(z)) for z in Z]
/tmp/ipykernel_95902/217849312.py:20: RuntimeWarning: invalid value encountered in subtract
  return max_val + np.log(np.mean(np.exp(data - max_val), axis=axis))


Theta optimal trouvé par l'optimisation: [ 2.  3.  2.  5.  5.  7.  6.  8.  9. 10.  9. 13. 13. 13. 15. 16. 16. 18.
 17. 20.]


Test avec ml_rr

In [11]:
def estimateur_ml_rr(theta, x):
    K = p.rvs(r)
    size = 2 ** (K + 1)
    Z = q_phi_z_given_x(x).rvs(size=size)
    Z_E, Z_O = Z[::2], Z[1::2]
    log_weights = [np.log(p_theta_z(theta).pdf(z) * p_theta_x_given_z(z).pdf(x) / q_phi_z_given_x(x).pdf(z)) for z in Z]
    log_weights_E, log_weights_O = log_weights[::2], log_weights[1::2]
    I_0 = np.mean(log_weights)
    l_hat_E, l_hat_O = logmeanexp(log_weights_E), logmeanexp(log_weights_O)
    l_hat_O_E = logmeanexp(log_weights)
    delta_k_list_rr = [logmeanexp(log_weights[:2**(k+1)]) - 0.5 * (logmeanexp(log_weights_O[:2**k]) + logmeanexp(log_weights_E[:2**k])) for k in range(K+1)]
    l_hat_ml_rr_x = I_0 + np.sum([delta_k_list_rr[k] / (1 - p(r).cdf(k-1)) for k in range(K+1)])
  
    
    
    
    # Retourner l'estimation
    return l_hat_ml_rr_x

def aggregate_estimator_ml_rr(theta):
    l_hat_ml_rr_list = []

    for x in X:
        l_hat_ml_rr_list.append(estimateur_ml_rr(theta, x))
    return np.mean(l_hat_ml_rr_list)

In [12]:
# Définir une fonction objectif pour l'optimisation
def objectif_ml_rr(theta):
    return -aggregate_estimator_ml_rr(theta)  # On minimise l'opposé de l'estimateur

# Supposer une valeur initiale pour theta
theta_init_ml_rr = np.random.randn(20)

# Minimiser la fonction objectif pour trouver theta optimal
resultat_optimisation_ml_rr = minimize(objectif_ml_rr, theta_init_ml_rr)

# Récupérer le theta optimal trouvé par l'optimisation
theta_optimal_ml_rr = resultat_optimisation_ml_rr.x

# Imprimer le résultat
print("Theta optimal trouvé par l'optimisation:", theta_optimal_ml_rr)

/tmp/ipykernel_95902/2570684156.py:6: RuntimeWarning: divide by zero encountered in log
  log_weights = [np.log(p_theta_z(theta).pdf(z) * p_theta_x_given_z(z).pdf(x) / q_phi_z_given_x(x).pdf(z)) for z in Z]
/tmp/ipykernel_95902/217849312.py:20: RuntimeWarning: invalid value encountered in subtract
  return max_val + np.log(np.mean(np.exp(data - max_val), axis=axis))


Theta optimal trouvé par l'optimisation: [ 1.06345614 -0.35839561  1.76785886  1.66490607  0.14678718 -0.10568308
 -1.41893123  1.07329356 -0.30465404  0.72262995  0.55217586 -0.2002613
  0.01808234  1.11868775 -0.133515   -0.33666163  0.27683162  0.22678559
  0.02206721 -0.9667478 ]


Test avec iwae

In [21]:
def estimateur_iwae(theta, x):
    K = p.rvs(r)
    size = 2 ** (K + 1)
    Z = q_phi_z_given_x(x).rvs(size=size)
    Z_E, Z_O = Z[::2], Z[1::2]
    log_weights = [np.log(p_theta_z(theta).pdf(z) * p_theta_x_given_z(z).pdf(x) / q_phi_z_given_x(x).pdf(z)) for z in Z]
    
    
    
    # Retourner l'estimation
    return logmeanexp(log_weights)

def aggregate_estimator_iwae(theta):
    l_hat_iwae_list = []

    for x in X:
        l_hat_iwae_list.append(estimateur_iwae(theta, x))
    return np.mean(l_hat_iwae_list)

In [22]:
# Définir une fonction objectif pour l'optimisation
def objectif_iwae(theta):
    return -aggregate_estimator_iwae(theta)  # On minimise l'opposé de l'estimateur

# Supposer une valeur initiale pour theta
theta_init_iwae = np.random.randn(20)

# Minimiser la fonction objectif pour trouver theta optimal
resultat_optimisation_iwae = minimize(objectif_iwae, theta_init_iwae)

# Récupérer le theta optimal trouvé par l'optimisation
theta_optimal_iwae = resultat_optimisation_iwae.x

# Imprimer le résultat
print("Theta optimal trouvé par l'optimisation:", theta_optimal_iwae)

/tmp/ipykernel_95902/2167204708.py:6: RuntimeWarning: divide by zero encountered in log
  log_weights = [np.log(p_theta_z(theta).pdf(z) * p_theta_x_given_z(z).pdf(x) / q_phi_z_given_x(x).pdf(z)) for z in Z]
/tmp/ipykernel_95902/217849312.py:20: RuntimeWarning: invalid value encountered in subtract
  return max_val + np.log(np.mean(np.exp(data - max_val), axis=axis))


Theta optimal trouvé par l'optimisation: [ 0.99066879 -0.12741301  1.93299073  0.65851622 -1.26168042 -0.37484456
  1.51298273 -1.39810348  1.03646724 -0.22213902  0.30820356  0.29121353
 -0.47940173  0.03726505  2.79478743  1.11376067  0.48151411  0.03756264
 -0.79826138 -0.78188891]


In [ ]:
Test avec Sumo

In [25]:
def estimateur_sumo(theta, x):
    K = p.rvs(r)
    size = 2 ** (K + 1)
    Z = q_phi_z_given_x(x).rvs(size=size)
    Z_E, Z_O = Z[::2], Z[1::2]
    log_weights = [np.log(p_theta_z(theta).pdf(z) * p_theta_x_given_z(z).pdf(x) / q_phi_z_given_x(x).pdf(z)) for z in Z]
    log_weights_E, log_weights_O = log_weights[::2], log_weights[1::2]
    I_0 = np.mean(log_weights)
    delta_k_list_sumo = [logmeanexp(log_weights[:2**(k+1)]) - logmeanexp(log_weights[:2**k]) for k in range(K+1)]
    l_hat_sumo_x = I_0 + np.sum([delta_k_list_sumo[k] / (1 - p(r).cdf(k-1)) for k in range(K+1)])
    
    

    return l_hat_sumo_x

def aggregate_estimator_sumo(theta):
    l_hat_sumo_list = []

    for x in X:
        l_hat_sumo_list.append(estimateur_sumo(theta, x))
    return np.mean(l_hat_sumo_list)

In [24]:
# Définir une fonction objectif pour l'optimisation
def objectif_sumo(theta):
    return -aggregate_estimator_sumo(theta)  # On minimise l'opposé de l'estimateur

# Supposer une valeur initiale pour theta
theta_init_sumo = np.random.randn(20)

# Minimiser la fonction objectif pour trouver theta optimal
resultat_optimisation_sumo = minimize(objectif_sumo, theta_init_sumo)

# Récupérer le theta optimal trouvé par l'optimisation
theta_optimal_sumo = resultat_optimisation_sumo.x

# Imprimer le résultat
print("Theta optimal trouvé par l'optimisation:", theta_optimal_sumo)

/tmp/ipykernel_95902/77341551.py:6: RuntimeWarning: divide by zero encountered in log
  log_weights = [np.log(p_theta_z(theta).pdf(z) * p_theta_x_given_z(z).pdf(x) / q_phi_z_given_x(x).pdf(z)) for z in Z]
/tmp/ipykernel_95902/217849312.py:20: RuntimeWarning: invalid value encountered in subtract
  return max_val + np.log(np.mean(np.exp(data - max_val), axis=axis))


NameError: name 'l_hat_sumo_list_x' is not defined